In [1]:
from IPython.display import Markdown as md
from random import randrange
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)
#  a hack to hide code from cell: https://github.com/damianavila/RISE/issues/32    

In [2]:
%%html
<style>
 .container.slides .celltoolbar, .container.slides .hide-in-slideshow {
    display: None ! important;
}
    
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
th, td {
  padding: 5px;
}
th {
  text-align: left;
}
</style>

In [3]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# Foreword 

The following code are used for this lecture
  * most of the code have been adapted from https://github.com/aimacode/aima-python

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    
def g(n): return n.path_cost    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]



def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [5]:
FIFOQueue = deque

LIFOQueue = list

StackAgenda = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [6]:
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [7]:
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

In [8]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(length=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} node extended|{:9,d} goal tested|{:5.0f} cost |{:8,d} length | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['length'], name))

In [9]:
# Some specific RouteProblems

romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})


r0 = RouteProblem('A', 'A', map=romania)
r1 = RouteProblem('A', 'B', map=romania)
r2 = RouteProblem('N', 'L', map=romania)
r3 = RouteProblem('E', 'T', map=romania)
r4 = RouteProblem('O', 'M', map=romania)
r5 = RouteProblem('S', 'E', map=romania)

In [10]:
def my_breadth_first_search(problem, use_extendedList = False):
    agenda = FIFOQueue([Node(problem.initial)])
#   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
                        agenda.appendleft(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.appendleft(child)
    return failure

In [11]:
# Python implementation of DFS
def my_depth_first_search(problem, use_extendedList= False):
    agenda = StackAgenda([Node(problem.initial)])
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       by default: pop() return the element at -1 (the end of the list)  
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       append at the end in Python has the same effect as adding at the front
#                       as pop() return from -1 by default
                        agenda.append(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.append(child)
    return failure

In [12]:
# Python implementation of uniform cost search
def my_uniform_cost_search(problem, use_extendedList= False):
    agenda = PriorityQueue([Node(problem.initial)], key=g)
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       PQ.pop() returns the node with the least cost 
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       PriorityQueue taks care of this by inserting to 
#                       an appropriate location that is efficient for further operation
                        agenda.add(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.add(child)
    return failure

# CS5010 Artificial Intelligence Principles
### Lecture 18 Informed Search 

Lei Fang

University of St Andrews

# Last week

* Uninformed searching problem

* A general searching algorithm template
  * BFS and DFS as specific cases
  
* Various searching algorithms based on the template
  * DFS
  * BFS
  * Depth limited search (tutorial)
  * Non-deterministic search
  * Uniform cost searching

# Recap: a general searching algorithm 


All algorithms we study follow a template
  * shouldn't be surprising: they all super-impose a search tree on the state space graph 
  * only differ in how the tree is constructed or super-imposed

Can be considered as the *mother* of all searching algorithms introduced here

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font> add <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> and reorganise <font color="purple"><i>agenda</i></font><br>   
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp;&nbsp;      # algorithms differ here!  <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>



# Variants of uninformed search

Differ in step 5:
* **DFS**: insert to the front of agenda
* **BFS**: insert to the back of the agenda 
* **Non-deterministic searching (NDS)**: randomly inject the newly extended paths to the existing agenda
* **Uniform cost search (UCS)**: insert then sort the agenda based on path cost
* **Depth limited search (DLS)**: reject all paths with length greather than the limit then insert


|DFS       |  BFS |      NDS|
|:-------------------------:|:-------------------------:| :--------------------------:|
|![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/dfsagenda.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfsagenda.png) | ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/nondagenda.png)|

# This time

* Informed search, aka Heuristic search
  * heuristics: extra information

* A few basic informed search algorithms 
  * Beam search
  * Greedy search
  * Hill climbing
  
  
* Next time: optimal search with heuristics
  * A* search
  * Heuristic functions 

# Uninformed search and Informed search

Take route finding problem as an example:

**Uninformed search**
* e.g. you are dropped to Mars and need to find the base-station
  * no extra information/heuristic at all
  * the best you can do is to blindly try all options available to you


**Informed search** also called *heuristic search* 
* e.g. from St Andrews to London  (assume you know  a tiny bit about the UK's geography)
  * Aberdeen or Shetland are probably not good choices for your next stop! you are making some *informed* search
  * Edinburgh makes better sense (as you know it is closer to London)

# Demonstration

* check Panopto video 

# Heuristics


A simple **heuristic** for route finding problem is, e.g.
  * **straight-line distance** between a node to the goal state (Bucharest), e.g.
  $$h_{SLD}(Arad) = 366$$
    * straight-distance between Arad and Bucharest is 366 miles
    * easily available and provides some sense of *direction*   
      * *Zerind* with a distance 374 probably is not a good choice to extend next!
    
    
    


|Romanian  map    |  Distance heuristic |   
|:-------------------------:|:-------------------------:|
|![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/distances.png) | 

# Best first search (BFS) (based on heuristic only) aka Greedy Search

Best First Search (BestFS) is very similar to Uniform Cost Search (UCS)
  * they both sort the entire **agenda** after insertion of the extended paths
  * the only difference is BFS sort agenda based on **heuristic** only rather than the actual cost
  * also known as Greedy search

Best First Search (based on heuristic only) pseudo-code:


<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> add <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> then sort the agenda based on heuristic</b><br>
&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;       # <b>yay ! another algorithm is "invented" </b> <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>

Note the only difference is which criteria is used to sort the whole agenda!

In [13]:
# Python implementation of best first search (based on heuristic only)
def my_best_first_search(problem, use_extendedList= False):
    agenda = PriorityQueue([Node(problem.initial)], key=problem.h)
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       PQ.pop() returns the node with the least cost 
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       PriorityQueue taks care of this by inserting to 
#                       an appropriate location that is efficient for further operation
                        agenda.add(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.add(child)
    return failure

In [14]:
print("The path found by Best First Search:", path_states(my_best_first_search(r1, use_extendedList=False)))
print(g(my_best_first_search(r1, use_extendedList=False)))

The path found by Best First Search: ['A', 'S', 'F', 'B']
450


# A quick walk-through

Skip initialisation step;

<font color='blue'><b>Step 1</b></font>:

Based on the heuristic, the next node to extended is Sibiu, or path [A,S]
  * it is the closest to the goal state Bucharest based on straight-line distance heuristic

--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romBst1.png)

--------------------------

<font color='blue'><b>Step 2</b></font>:

After insertion, resort the agenda based on the heuristic, the next to pop, check and extend is *Fagaras*

--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romBst2_.png)

--------------------------

<font color='blue'><b>Step 3</b></font>:

Within 3 steps, we have found the solution !

The next one to be popped and checked is the destination 
  * with a distance to itself zero
  
So BestFS is not optimal but seems very *straight to the point*
  * so given the name Greedy search 
  * make greedy decision each round

--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romBst3_.png)

--------------------------

# Demonstration of Best First Search

* Check Panopto video
* the search is no longer *clueless* but very *directed*

# Hill Climbing 


If you do not want to sort the **whole** agenda, an alternative is 
  * to sort only the **newly extended paths** 
  * and add the sorted paths to the front of the **agenda**


Note the difference to Greedy or Best first search
  * it only sort the newly extended paths not the whole agenda !

The algorithm is called **Hill Climbing**
  * similar to DFS (adding to the front)
  * but with the help of heuristic 
  * basically the informed version of DFS

Hill Climbing (based on heuristic only) pseudo-code:


<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> sort <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> based on heuristic then add them to the front of <font color="purple"><i>agenda</i></font></b><br>
&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;       # <b>yay ! another algorithm is "invented" </b> <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>

Note the only difference is which criteria is used to sort the whole agenda!

In [15]:
# Python implementation of best first search (based on heuristic only)
def my_hill_climbing_search(problem, use_extendedList= False):
    agenda = StackAgenda([Node(problem.initial)])
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       by default: pop() return the element at -1 (the end of the list)  
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                children = []
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       append at the end in Python has the same effect as adding at the front
#                       as pop() return from -1 by default
                        children.append(child)
                children.sort(reverse=True, key=problem.h)
                agenda.extend(children)
        else:
            children = []
            for child in expand(problem, node):
                if not is_cycle(child):
                    children.append(child)
            children.sort(reverse=True, key=problem.h)
            agenda.extend(children)
    return failure

# Demo of Hill Climbing

Note the difference between DFS and Hill Climbing
  * with the help of heuristic, Hill Climbing no longer finds the detoured route

In [16]:
print("The path found by UCS is:",path_states(my_uniform_cost_search(r1, use_extendedList=False)))
print("The path found by DFS is:", path_states(my_depth_first_search(r1, use_extendedList=False)))
print("The path found by Hill Climbing is:", path_states(my_hill_climbing_search(r1, use_extendedList=False)))

print("The cost of the path found by UCS is:",g(my_uniform_cost_search(r1, use_extendedList=False)))
print("The cost of the path found by DFS is:", g(my_depth_first_search(r1, use_extendedList=False)))
print("The cost of the path found by Hill Climbing is:", g(my_hill_climbing_search(r1, use_extendedList=False)))

The path found by UCS is: ['A', 'S', 'R', 'P', 'B']
The path found by DFS is: ['A', 'T', 'L', 'M', 'D', 'C', 'P', 'B']
The path found by Hill Climbing is: ['A', 'S', 'F', 'B']
The cost of the path found by UCS is: 418
The cost of the path found by DFS is: 733
The cost of the path found by Hill Climbing is: 450


# Why called Hill Climbing ?
   

Hill Climbing is very similar to DFS
  * new **(sorted)** paths are added to the front of agenda
  * it inherits most properties of DFS  
    * go as far as possible 
    * but guided by heuristic at each location
  

As a result, the searching behaves like climbing a **hill** that is closest to the starting point
  * go as far as possible 
    * at each junction/location, choose the best next direction (climb a hill of the current branch)
  * no return to check another hill might be better 
  * what it find might be local optimum !

If the state space is continuous, **Hill Climbing**'s equivalence is **Gradient Ascent**
  * solution found by Hill Climbing depends on the starting point
  * likely to return a local optimum rather than the optimal 
  
Remedy is to restart the search a few times from multiple random intial positions
  * return the optimal among optimals

![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/hillclimbing.png)

In [17]:
report([my_breadth_first_search, my_depth_first_search, my_uniform_cost_search, my_best_first_search, my_hill_climbing_search], [r1,r2,r3,r4,r5])  

my_breadth_first_search:
       34 node extended|       14 goal tested|  450 cost |       3 length | RouteProblem('A', 'B')
       83 node extended|       34 goal tested| 1085 cost |       9 length | RouteProblem('N', 'L')
       55 node extended|       23 goal tested|  837 cost |       7 length | RouteProblem('E', 'T')
       77 node extended|       29 goal tested|  445 cost |       5 length | RouteProblem('O', 'M')
      112 node extended|       47 goal tested|  579 cost |       5 length | RouteProblem('S', 'E')
      361 node extended|      147 goal tested| 3396 cost |      29 length | TOTAL

my_depth_first_search:
       17 node extended|        8 goal tested|  733 cost |       7 length | RouteProblem('A', 'B')
       36 node extended|       16 goal tested| 1240 cost |      11 length | RouteProblem('N', 'L')
       29 node extended|       12 goal tested|  992 cost |       9 length | RouteProblem('E', 'T')
       35 node extended|       16 goal tested|  895 cost |       7 length | R

# Beam Search


Beam search is the informed version of Breadth first search (BFS)
  * visit nodes based on the number of hops away from the root: $l$ tiers first then $l+1$
  * but is it significantly more space efficient

It achieves this by trimming the search tree based on heuristic 

It requires an extra parameter called **beam size**: K
  * trim less optimal paths early on 

Specifically, Beam Search's agenda reorganisation step (step 5) is:

* only keep the **best K paths** of each tier $l$


So the space complexity if linear $O(kd)$

![](https://ars.els-cdn.com/content/image/1-s2.0-S0377221798003191-gr1.gif)

Beam Search pseudo-code:


<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> add <font color="purple"><i>new paths</i></font> (the length of new paths is $l$) from step <font color="blue">4</font> to the back of <font color="purple"><i>agenda</i></font>; only keep the best K path of length $l$ based on heuristic</b><br>
&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;       # <b>yay ! another algorithm is "invented" </b> <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>

At each tier, we have only $K$ best paths left in agenda!

# Summary

* Informed searching algorithm variants
  * Greedy search or Best first search (informed version of UCS)
  * Hill climbing (informed version of DFS)
  * Beam search (informed version of BFS)